In [ ]:
import numpy as np
from scipy.sparse import diags, csc_matrix, save_npz
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from time import time
from scipy import io

from assimulo.solvers import KINSOL
from assimulo.problem import Algebraic_Problem


%matplotlib inline

In [ ]:
def f(x, S, N):
    #  Constants
    m = 70
    R0 = 6370000
    H = 8000
    
    y = np.zeros( x.shape )
    y[0] = 32000
    
    y[N:] = -1/(2 * m) * np.exp(-x[0:N] / H) * x[N:] * np.abs(x[N:]) - \
    9.8 / np.square(1 - x[0:N] / R0)
    
    y[N] = 0
    
    res = S * x -y
    
    return res

In [ ]:
t = np.arange(0,385, 0.1)
N = t.shape[0]
Delta = np.diff(t)[0]
print(N)
x = np.ones(2 *N)

In [ ]:
#  Construct sprase matrix
d = np.ones(2 * N) / Delta
dm1 = -np.ones( 2 * N - 1 ) / Delta
dN = -np.ones( N )

k = [-1, 0, N]
S = diags([dm1, d, dN], k, format = 'csc')
S[0, 0] = 1
S[N, N - 1] = 0

In [ ]:
args = (S, N)

#start = time()
#x = fsolve(f, x, args = args)
#end = time()

In [ ]:
#plt.plot(t, x[0:N])

In [ ]:
#plt.plot(t, x[N:])

In [ ]:
class Problem:
    def __init__(self, S, N):
        self.S = S
        self.N = N
        
        x = np.ones(2 * self.N)
        
        alg_mod = Algebraic_Problem(self.f, y0=x)
    
        #Define the KINSOL solver
        alg_solver = KINSOL(alg_mod)
    
        #Solve
        self.y = alg_solver.solve()
        
    def f(self, x):
        #  Constants
        m = 70
        R0 = 6370000
        H = 8000
    
        y = np.zeros( x.shape )
        y[0] = 32000
    
        y[self.N:] = -1/(2 * m) * np.exp(-x[0:self.N] / H) * x[self.N:] * np.abs(x[self.N:]) - \
        9.8 / np.square(1 - x[0:self.N] / R0)
    
        y[self.N] = 0
    
        res = self.S * x -y
    
        return res

In [ ]:
p = Problem(S, N)